In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats as stats
from scipy.stats import gaussian_kde as gkde
from bokeh.plotting import figure, show, output_notebook, gridplot
from bokeh.models import Span

output_notebook()

Loading BokehJS ...

In [2]:
def normPDF(x, mu=0.0, sig2=1.0, prec=None):
    if prec:
        sig2 = 1.0 / prec
    return stats.norm.pdf(x, mu, np.sqrt(sig2))

def normRVS(samples=1, mu=0.0, sig2=1.0, prec=None):
    if prec:
        sig2 = 1.0 / prec
    return np.random.normal(mu, np.sqrt(sig2), size=samples)

def gammaRVS(alpha, beta, samples=1):
    return np.random.gamma(alpha, beta, size=samples)

def invGammaRVS(alpha, beta, samples=1):
    return beta * sp.stats.invgamma.rvs(alpha, size=samples)

def invGammaPDF(x, alpha, beta=1.0):
    # According to:
    # http://reference.wolfram.com/language/ref/InverseGammaDistribution.html
    # beta is the scale parameter.
    return sp.stats.invgamma.pdf(x, alpha, scale=beta)

In [3]:
def plotMCMCResults(xs, rx, ys, ry, title="Markov-chain results"):
    t = np.arange(xs.shape[0])
    f = figure(title=title, x_axis_label='t',
               plot_width=400, plot_height=400)
    f.line(t, xs, color="darkgreen", alpha=0.5)
    realX = Span(location=rx, dimension='width', 
                 line_color='firebrick', line_dash='dashed',
                 line_width=2)
    f.add_layout(realX)
    f.line(t, ys, color="firebrick", alpha=0.5)
    realY = Span(location=ry, dimension='width', 
                 line_color='darkgreen', line_dash='dashed',
                 line_width=2)
    f.add_layout(realY)
    show(f)

## Mean and precision of a Gaussian distribution

Find the mean ($\mu$) and precision ($\tau = 1/\sigma^2$) of a univariate Gaussian distribution using different approaches.

In [4]:
N = 23
mu = 3.0
sigma2 = 0.5
prec = 1/sigma2
print("mu = {}, sigma2 = {} and tau = {}".format(mu,sigma2,prec))
X = np.linspace(-3.0, 5.0, N)
PX = [normPDF(x,mu,prec=prec) for x in X]
f = figure(title="Real distribution",
           x_axis_label='x',
           y_axis_label='p(x)',
           plot_width=300, plot_height=300)

f.line(X, PX, line_width=2, color="firebrick")
show(f)

mu = 3.0, sigma2 = 0.5 and tau = 2.0


The following code produces the set $D=\{x_n\}_{n=1}^N$ of samples produced by the distribution.

In [5]:
D = normRVS(samples=N, mu=mu, prec=prec)

- $N = |D|$
- $\hat{\mu}_{ML} = \frac{1}{N}\sum_{i=1}^{N} x_i$
- $\hat{\sigma}^2_{ML} = \frac{1}{N}\sum_{i=1}^{N} (x_i - \bar{x})^2$

In [6]:
# MLE estimators 
sigma2ML = np.var(D)
muML = np.mean(D)

### Metropolis-Hastings

The idea is to sample from $p(D | \mu, \tau)\times p(\mu,\tau)$. When the Markov chain is at its stationary state then we will be sampling from the posterior $p(\mu, \tau | D)$. As prior we use the normal-gamma distribution on the parameters $\mu$ and $\tau$.

\begin{align*}
p(\mu, \tau | D) &= \frac{p(D | \mu, \tau)\times p(\mu,\tau)}{p(D)} \\
& \propto p(D | \mu, \tau)\times p(\mu,\tau) \\
\end{align*}

- Likelihood:

\begin{align*}
p(D \vert \mu,\tau) = \frac{1}{(2\pi)^{N/2}} \tau^{N/2} exp \left\{- \frac{\tau}{2} \sum_{i=1}^N (x_i - \mu)^2\right\}
\end{align*}

- Prior:

\begin{align*}
p(\mu,\tau) = NGamm(\mu,\tau \vert \mu_0, \alpha_0, \beta_0) = \mathcal{N}\left(\mu \vert \mu_0, \tau^{-1}\right) Gamm(\tau \vert \alpha_0, \beta_0)
\end{align*}

In [7]:
def likelihood(mu,tau):
    a = 1.0 / ((2.0 * np.pi) ** (N / 2.0))
    b = tau ** (N / 2.0)
    c = - (tau / 2.0) * np.sum((D - mu) ** 2)
    return a * b * np.exp(c)

Hyper parameters

In [8]:
mu0 = 5
a0 = 1.0
b0 = 15.0
k0 = 2.0
r0 = 1.0

In [9]:
def Zng():
    # Computes the normalization constant of the normal-gamma distribution
    ft = sp.special.gamma(a0) / (b0 **  a0)
    return ft * ((2.0 * np.pi) / k0) ** (0.5)

def norm_gamma(mu, tau):
    zng = Zng()
    a = tau ** (a0 - 0.5)
    return (1.0 / zng) * a * np.exp(-(tau / 2.0) * (k0 * ((mu - mu0) ** 2) + 2.0 * b0))

def prior(mu, tau):
    return norm_gamma(mu, tau)

In [10]:
def Draw_sigma():
    alpha = a0 + N / 2.0
    coef = r0 / (r0 + N)
    beta = b0 + (N / 2.0) * (sigma2ML + coef*(muML - mu0)**2)
    sigma2_sample = invGammaRVS(alpha, beta)
    return sigma2_sample

def Draw_mu_given_sigma(sigma):
    distMean = (r0 * mu0 + N * muML) / (r0 + N)
    distSigma = np.sqrt(sigma / (r0 + N))
    mu_sample = sp.stats.norm.rvs(distMean, scale=distSigma)
    return mu_sample

def DrawQ():
    sigma2_sample = Draw_sigma()
    mu_sample = Draw_mu_given_sigma(sigma2_sample)
    return mu_sample, (1 / sigma2_sample)

In [11]:
# Computes the proportion between the posterior probability with the new
# parameters mu and tau and the posterior probability of the *currently* 
# accepted values mut and taut.
def r(mu, tau, mut, taut, u):
    num = prior(mu, tau) * likelihood(mu, tau)
    den = prior(mut, taut) * likelihood(mut, taut)
    return num > den * u

def acceptance(mu, tau, mut, taut, u):
    return r(mu, tau, mut, tau, u)

def MetropolisHastings(iter):
    mut = 10
    taut = 10
    mu = np.zeros(iter)
    sig = np.zeros(iter)
    acc = 0
    for i in range(iter):
        mus, taus = DrawQ()
        u = np.random.uniform()
        a = acceptance(mus, taus, mut, taut, u)
        if a:
            mut, taut = mus, taus
            acc = acc + 1
        mu[i] = mut
        sig[i] = taut
    
    return mu, sig

In [12]:
iterations = 10000
emu, etau = MetropolisHastings(iterations)

In [13]:
plotMCMCResults(emu, mu, etau, sigma2)

In [14]:
burnIn = 0.5
thin = 10
start = int(np.size(emu) * burnIn)

# Apply burn-in
emu = emu[start:]
etau = etau[start:]
# Apply thin
idxs = np.arange(0,emu.size,thin)
emu = emu[idxs]
etau = etau[idxs]

In [15]:
plotMCMCResults(emu, mu, etau, sigma2)

#### Conclusions

The plots above show the convergence of the Markov chain. The first plot presents the values of $\mu$ and $\sigma^2$ at every iteration of the program (every time $t$). It can be appreciated from this plot that the chain iterates on values close to both parameters. The real value of each parameter is depicted as a dotted line.

The second plot shows the chain after the burn in where $50\%$ of the data is discarded. Then in the "thin" phase we choose one every ten samples hoping that they are independent samples from the actual posterior distribution.

During the development of the algorithms we tried different proposal distributions. It was the one with the normal-gamma that exhibited the best results. For the others it was "easy" to derive the mean but they did not manage to derive the variance. This might be due to the dependency between the two distributions (the normal and the gamma). Sampling from the distributions is a two step process, first we draw a sample for $\sigma^2$ from the gamma and use that one to draw a sample from the normal.

We observed that is quite important to use the maximum likelihood estimators for both parameters when computing the value of each of the distributions parameters. This is actually where the data $D$ is taken into account to sample from the posterior.

#### References

- http://www.inference.phy.cam.ac.uk/itprnn/book.pdf
- http://stats.stackexchange.com/questions/137710/metropolis-hastings-using-log-of-the-density
- https://www.youtube.com/watch?v=OTO1DygELpY